<a href="https://colab.research.google.com/github/greed4040/Ziff-deep-learning/blob/master/Localization_with_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object Detection with Bounding Box Regression

This notebook will demonstrate some of the basic steps in object detection. Bounding box regression, as the name suggests, makes a regressor model which directly outputs the bounding box coordinates given an image.

This concept has been internal to object detectors like [YOLO](https://arxiv.org/abs/1506.02640).

Note: If you are running this notebook on Colab itself, I insist you turn on the GPU runtime. Go to **Tools ~> Runtime ~> Change runtime type ~> Hardware Accelerator ~> Select "GPU"**

Let's Start!




## 1) Importing the packages

We import TensorFlow and Keras along with our beloved NumPy. We print the TensorFlow version we're using.


In [1]:

%tensorflow_version 2.x

from tensorflow import keras
from tensorflow.keras import backend as K
import tensorflow as tf
import numpy as np


TensorFlow 2.x selected.


## 2) Downloading the data from GitHub

We need to download our dataset from the [Dataset_Archives](https://github.com/shubham0204/Dataset_Archives) repo where I have hosted a ZIP file of the dataset. The original dataset hails from [Kaggle.com](https://www.kaggle.com) as [Image Localization Dataset](https://www.kaggle.com/mbkinaci/image-localization-dataset) by [Muhammed Buyukkinaci](https://www.kaggle.com/mbkinaci).

Also, we install the [xmltodict](https://pypi.org/project/xmltodict/) package from PyPI which will be useful to parse the XML files containing the bounding box annotations.


In [2]:

!pip install xmltodict
!wget https://github.com/shubham0204/Dataset_Archives/blob/master/image-localization-dataset.zip?raw=true -O data.zip
!unzip data.zip


--2020-03-10 10:33:47--  https://github.com/shubham0204/Dataset_Archives/blob/master/image-localization-dataset.zip?raw=true
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/shubham0204/Dataset_Archives/raw/master/image-localization-dataset.zip [following]
--2020-03-10 10:33:47--  https://github.com/shubham0204/Dataset_Archives/raw/master/image-localization-dataset.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shubham0204/Dataset_Archives/master/image-localization-dataset.zip [following]
--2020-03-10 10:33:48--  https://raw.githubusercontent.com/shubham0204/Dataset_Archives/master/image-localization-dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connectin

## 4) Parsing the images from the dataset

We prepare a list of all the files under `training_images` folder which have a `*.jpg` extension ( image files ) using `glob`. We resize them to the `input_dim` and normalize them.

In [0]:

from PIL import Image , ImageDraw
import os
import glob

input_dim = 228

images = []
image_paths = glob.glob( 'training_images/*.jpg' )
for imagefile in image_paths:
    image = Image.open( imagefile ).resize( ( input_dim , input_dim ))
    image = np.asarray( image ) / 255.0
    images.append( image )


## 5) Parsing the bounding box annotations

We parse all XML files under the `training_images` folder. These annotations are in the popular `PASCAL-VOC` format. We extract the four bounding box coordinates and normalize them using `input_dim`.

In [0]:

import xmltodict
import os

bboxes = []
annotations_paths = glob.glob( 'training_images/*.xml' )
for xmlfile in annotations_paths:
    x = xmltodict.parse( open( xmlfile , 'rb' ) )
    bndbox = x[ 'annotation' ][ 'object' ][ 'bndbox' ]
    bndbox = np.array([ int(bndbox[ 'xmin' ]) , int(bndbox[ 'ymin' ]) , int(bndbox[ 'xmax' ]) , int(bndbox[ 'ymax' ]) ])
    bboxes.append( bndbox / input_dim )



## 6) Preparing the final datasets

Our input will be an array of all the images we parsed earlier. The target output will be the bounding box coordinates.

Using scikit-learn's `train_test_split` method, we split our dataset in training and validation datasets.



In [13]:

Y = np.array( bboxes )
X = np.array( images )

Y = np.reshape( Y , ( -1 , 1 , 1 , 4 ) )

print( X.shape ) 
print( Y.shape )

x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.1 )


(186, 228, 228, 3)
(186, 1, 1, 4)


NameError: ignored

## 7) Defining the loss function and evaluation metrics

First, we define a method which calculates the intersection over union ( IOU ) of two bounding boxes. In the `custom_loss`, we first calculate Mean Squared Error of the target and predicted bounding boxes. Next, we calculate the IOU loss which is $1 - iou$.

Hence, our final loss function is like,

$\Large L( y , y' ) = MSE( y, y') + ( 1 - IOU( y , y' ))$

Also, we use IOU as a validation metric to evaluate the model's performance on the testing dataset.

**Note: The loss function is a custom implementation. IOU is mostly used as a metric and not in the loss function.**



In [0]:

input_shape = ( input_dim , input_dim , 3 )

def calculate_iou( target_boxes , pred_boxes ):
    xA = K.maximum( target_boxes[ ... , 0], pred_boxes[ ... , 0] )
    yA = K.maximum( target_boxes[ ... , 1], pred_boxes[ ... , 1] )
    xB = K.minimum( target_boxes[ ... , 2], pred_boxes[ ... , 2] )
    yB = K.minimum( target_boxes[ ... , 3], pred_boxes[ ... , 3] )
    interArea = K.maximum( 0.0 , xB - xA ) * K.maximum( 0.0 , yB - yA )
    boxAArea = (target_boxes[ ... , 2] - target_boxes[ ... , 0]) * (target_boxes[ ... , 3] - target_boxes[ ... , 1])
    boxBArea = (pred_boxes[ ... , 2] - pred_boxes[ ... , 0]) * (pred_boxes[ ... , 3] - pred_boxes[ ... , 1])
    iou = interArea / ( boxAArea + boxBArea - interArea )
    return iou

def custom_loss( y_true , y_pred ):
    mse = tf.losses.mean_squared_error( y_true , y_pred ) 
    iou = calculate_iou( y_true , y_pred ) 
    return mse + ( 1 - iou )

def iou_metric( y_true , y_pred ):
    return calculate_iou( y_true , y_pred ) 
    

## 8) Making the CNN model

We'll use a CNN model which takes in input an image of shape `( input_dim , input_dim , 3 )` and outputs a vector of shape `( 1 , 1 , 4 )`.

*   The `Conv2D` layers will extract features from the image.
*   We will not use `Dense` layers as they degrade the performance of our model.

We use a small learning rate of 0.0001 so that the learning doesn't stop. Also, we pass the `iou_metric` in the `model.compile()` method.



# New Section

In [0]:

model_layers = [       
    keras.layers.Conv2D( 256 , input_shape=( input_dim , input_dim , 3 ) , kernel_size=( 3 , 3 ) , strides=2 , activation='relu' ),
    keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=2 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 64 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 64 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 64 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 64 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D( 16 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 16 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' ),

    keras.layers.Conv2D( 4 , kernel_size=( 2 , 2 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 4 , kernel_size=( 2 , 2 ) , strides=1 , activation='relu' ),
    keras.layers.Conv2D( 4 , kernel_size=( 2 , 2 ) , strides=1 , activation='sigmoid' ),
]

model = keras.Sequential( model_layers )
model.compile(
	optimizer=keras.optimizers.Adam( lr=0.0001 ),
	loss=custom_loss,
    metrics=[ iou_metric ]
)
model.summary()


## 9) Training the model

We finally train the model with the validation dataset and save it to the disk.

In [0]:

model.fit( 
    x_train ,
    y_train , 
    validation_data=( x_test , y_test ),
    epochs=50 ,
    batch_size=5 
)

model.save( 'model.h5')


## 10) Testing the model on test images

The following code will predict bounding boxes for some unseen images, draw them on the image using `ImageDraw` and then save them to a folder ( created with `mkdir` command ).

In [0]:

!rm -r inference_images
!mkdir -v inference_images

boxes = model.predict( x_test )
for i in range( boxes.shape[0] ):
    b = boxes[ i , 0 , 0 , 0 : 4 ] * input_dim
    img = x_test[i] * 255
    source_img = Image.fromarray( img.astype( np.uint8 ) , 'RGB' )
    draw = ImageDraw.Draw( source_img )
    draw.rectangle( b , outline="black" )
    source_img.save( 'inference_images/image_{}.png'.format( i + 1 ) , 'png' )


## 11) Print the average IOU

We will calculate the average IOU score over the validation dataset.

In [0]:

def calculate_avg_iou( target_boxes , pred_boxes ):
    xA = np.maximum( target_boxes[ ... , 0], pred_boxes[ ... , 0] )
    yA = np.maximum( target_boxes[ ... , 1], pred_boxes[ ... , 1] )
    xB = np.minimum( target_boxes[ ... , 2], pred_boxes[ ... , 2] )
    yB = np.minimum( target_boxes[ ... , 3], pred_boxes[ ... , 3] )
    interArea = np.maximum(0.0, xB - xA ) * np.maximum(0.0, yB - yA )
    boxAArea = (target_boxes[ ... , 2] - target_boxes[ ... , 0]) * (target_boxes[ ... , 3] - target_boxes[ ... , 1])
    boxBArea = (pred_boxes[ ... , 2] - pred_boxes[ ... , 0]) * (pred_boxes[ ... , 3] - pred_boxes[ ... , 1])
    iou = interArea / ( boxAArea + boxBArea - interArea )
    return iou

target_boxes = y_test * input_dim
pred = model.predict( x_test )
pred_boxes = pred[ ... , 0 : 4 ] * input_dim

iou_scores = calculate_avg_iou( target_boxes , pred_boxes )
print( 'Mean IOU score {}'.format( iou_scores.mean() ) )


## ( Optional ) Convert the Keras model to TensorFlow Lite model

We can convert the Keras models ( .h5 ) to a TensorFlow Lite model ( .tflite ) for using it with Android or iOS devices.


In [0]:

converter = tf.lite.TFLiteConverter.from_keras_model_file( 'model.h5' , custom_objects={ 'custom_loss' : custom_loss , 'iou_metric' : iou_metric } ) 
converter.post_training_quantize = True
buffer = converter.convert()
open( 'model.tflite' , 'wb' ).write( buffer )
